REF: https://www.youtube.com/watch?v=JkiVKTqTK18

In [1]:
# 1. importamos las librerias
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
from ib_insync import *
util.startLoop()


In [2]:
# 2. Definimos una clase IB API para hacer todo lo que necesitemos
class IBapi(EWrapper, EClient):
	
	# carga de funciones iniciales de la API de IB
	def __init__(self):
		EClient.__init__(self, self)

	# solicita un ID para acceder a la API
	def nextValidId(self, orderId: int):
		super().nextValidId(orderId)
		self.nextorderId = orderId
		print('The next valid order id is: ', self.nextorderId)

	# las tres sisguentes funciones solo pintan información sobre la orden que queramos ejecutar
	def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
		print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining, 'lastFillPrice', lastFillPrice)
	
	def openOrder(self, orderId, contract, order, orderState):
		print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action, order.orderType, order.totalQuantity, orderState.status)

	def execDetails(self, reqId, contract, execution):
		print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId, execution.orderId, execution.shares, execution.lastLiquidity)

In [3]:
# 3. Definimos funciones utilies para la API
def run_loop():
	app.run()
 

def defineContract(symbol,secType,exchange,currency='USD'):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency
    return contract

def createOrder(action,totalQuantity,orderType,lmtPrice=False):
    order = Order()
    order.action = action
    order.totalQuantity = totalQuantity
    order.orderType = orderType
    if orderType == 'LMT':
        order.lmtPrice = lmtPrice
    return order

In [4]:
# 4. Ejecutamos el programa

# 4.1 nos conectamos a la API de IB
app = IBapi()
app.connect('127.0.0.1', port = 7497, clientId = 0)
app.nextorderId = None # Definimos el ID por defecto como 'None'
    
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

# 4.2.- esperamos sin hacer nada hasta que se nos asigna una ID
while True:
	if isinstance(app.nextorderId, int):
		print('connected')
		break
	else:
		print('waiting for connection')
		time.sleep(1)
  
# 4.3.- una vez asignada la ID tenemos que crear 2 objetos, uno para el contrato y otro para el tipo de orden

# Definimos el instrumento financiero que queremos, este caso el par GBP/USD
contract = defineContract(symbol='GBP',secType='CASH',exchange='IDEALPRO')

# Compramos 25000 libras
order = createOrder(action='BUY',totalQuantity=25000,orderType='LMT',lmtPrice='1.36')

app.placeOrder(app.nextorderId, contract, order)
time.sleep(3)

print('cancelling order')
app.cancelOrder(app.nextorderId)

time.sleep(3)
app.disconnect()

ERROR -1 2104 La conexión al centro de datos funciona correctamente:usfuture
ERROR -1 2104 La conexión al centro de datos funciona correctamente:eufarm
ERROR -1 2104 La conexión al centro de datos funciona correctamente:cashfarm
ERROR -1 2104 La conexión al centro de datos funciona correctamente:usfarm
ERROR -1 2106 La conexión al centro de datos HMDS funciona correctamente:euhmds
ERROR -1 2106 La conexión al centro de datos HMDS funciona correctamente:cashhmds
ERROR -1 2106 La conexión al centro de datos HMDS funciona correctamente:ushmds
ERROR -1 2158 La conexión de la granja de datos "sec-def" funciona correctamente:secdefeu


waiting for connection
The next valid order id is:  4
connected
openOrder id: 4 GBP CASH @ IDEALPRO : BUY LMT 25000.0 Submitted
orderStatus - orderid: 4 status: Submitted filled 0.0 remaining 25000.0 lastFillPrice 0.0
Order Executed:  -1 GBP CASH USD 000132b0.67a3f060.01.01 4 25000.0 2
openOrder id: 4 GBP CASH @ IDEALPRO : BUY LMT 25000.0 Filled
orderStatus - orderid: 4 status: Filled filled 25000.0 remaining 0.0 lastFillPrice 1.2376
openOrder id: 4 GBP CASH @ IDEALPRO : BUY LMT 25000.0 Filled
orderStatus - orderid: 4 status: Filled filled 25000.0 remaining 0.0 lastFillPrice 1.2376


ERROR 4 10148 La OrderId 4 que debe cancelarse no puede cancelarse; estado: Filled.


cancelling order


: 